In [8]:
import os
import vitaldb
import numpy as np
import pandas as pd
import scipy.signal as sig
from scipy.signal import welch, savgol_filter
import csv
import matplotlib.pyplot as plt
from scipy.integrate import simps
from mne.time_frequency import psd_array_multitaper
import neurokit2 as nk
import warnings
warnings.filterwarnings(action='ignore')
def consecutive(data, stepsize=0, threshold=100):
    con_list = np.split(data, np.where(np.diff(data) != stepsize)[0]+1)
    con_count = [d for d in con_list if len(d)>threshold]
    return con_count

# searching available cases & collect intubation information
available_casedf = pd.read_csv("https://api.vitaldb.net/cases")
cases = available_casedf['caseid'].values
evtdf = pd.read_csv('Supplementary Table 1.csv', header=3)

# parameters for extraction
datatrks = ['EVENT', 'NIBP_SBP','ART_SBP','HR','PPF20_CE','RFTN20_CE','EEG1_WAV', 'BIS'] 
eegcol = 'EEG1_WAV'
hrcol = 'HR'
ppfcol = 'PPF20_CE'
remicol = 'RFTN20_CE'

# settings for extraction 
MAX = 9999
SRATE = 128
SEGLENSEC = 30
OFFSETSEC = 10
SEGLEN = int(SRATE * SEGLENSEC)
SLIDE = int(SRATE * OFFSETSEC)
DELTA_LOW = 1
DELTA_HIGH = 4
THETA_LOW = 4
THETA_HIGH = 8
ALPHA_LOW = 8
ALPHA_HIGH = 12
BETA_LOW = 12
BETA_HIGH = 25

prewindow = int(1 * 60 * SRATE) # analyze 1 minute before intubation
pstwindow = int(1 * 60 * SRATE) # analyze 1 minute after intubation
nibp_window = int(2.5 * 60 * SRATE) # exception for non-invasive blood pressure measurements taken at 2.5 minute intervals


### parameters for result
freqs = np.linspace(0.0, 64.0, int(64*SEGLENSEC+1))
freqcols = list(map(lambda f: str(round(f, 2)) + 'Hz', freqs))
norm_freqcols = list(map(lambda f: 'norm_'+str(round(f, 2)) + 'Hz', freqs))
filecols = ['vitalfile', 'intu_dur','hr', 'bp','hr_change', 'bp_change', 'timing', 'PPF20_CE','RFTN20_CE', 'BIS',\
            'total', 'delta', 'theta', 'alpha', 'beta', 'delta_rel', 'theta_rel', 'alpha_rel', 'beta_rel',\
                  *freqcols, *norm_freqcols]

RESULT_FILE = f'result.csv'
with open(RESULT_FILE, 'a', newline='', encoding='utf-8-sig') as f:
    wr = csv.writer(f)
    wr.writerow(filecols)


### EEG analysis
def process_eeg(eegdf, ix, window, seg, slide, prev_row):    
    for i in range(0, window-seg, slide):
        temp = []  

        small_eeg = eegdf.iloc[ix+i:ix+i+seg]

        if len(small_eeg) < seg/2:
            continue

        if small_eeg.isnull().mean()>0.05:
            continue

        small_eeg = small_eeg.fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values   

        small_eeg -= savgol_filter(small_eeg, 51, 3)

        nall = len(consecutive(small_eeg, stepsize=0, threshold=SRATE*1))
        if nall > 1:
            continue

        psd, freqs = psd_array_multitaper(small_eeg, SRATE, adaptive=True, normalization='full', verbose=0)  # https://raphaelvallat.com/bandpower.html
        fres = freqs[1] - freqs[0]
    
        total = simps(psd, dx=fres)      
        
        delta = simps(psd[(freqs >= DELTA_LOW) & (freqs <= DELTA_HIGH)], dx=fres)
        theta = simps(psd[(freqs >= THETA_LOW) & (freqs <= THETA_HIGH)], dx=fres)
        alpha = simps(psd[(freqs >= ALPHA_LOW) & (freqs <= ALPHA_HIGH)], dx=fres)
        beta = simps(psd[(freqs >= BETA_LOW) & (freqs <= BETA_HIGH)], dx=fres)
    
        delta_rel = 10*np.log(delta/total) 
        theta_rel = 10*np.log(theta/total) 
        alpha_rel = 10*np.log(alpha/total) 
        beta_rel = 10*np.log(beta/total) 
        total = simps(psd, dx=fres)

        bandpower = {'total':total,'delta':delta,'theta':theta,'alpha':alpha,'beta':beta,\
                    'delta_rel':delta_rel,'theta_rel':theta_rel,'alpha_rel':alpha_rel,'beta_rel':beta_rel, 'i':i}
     

        norm_psd = psd /np.sum(psd)
        norm_psd = 10* np.log(norm_psd)

        power_dict = {f: p for f, p in zip(freqcols, psd)}
        norm_power_dict = {'norm_'+f: p for f, p in zip(freqcols, norm_psd)}

        temp.append({**bandpower, **power_dict, **norm_power_dict})

        temp = pd.DataFrame(temp)

        if np.isnan(delta):
            continue

        row = {**prev_row, **temp}     
        
        rowdf = pd.DataFrame(row, columns=filecols, index=[0])
        
        with open(RESULT_FILE, 'a', newline='', encoding='utf-8-sig') as f:
            wr = csv.writer(f)
            wr.writerow(rowdf.values[0])
       


for icase in cases[:MAX]:

    vf = vitaldb.load_case(icase, datatrks, 1 / SRATE)
    file = '{:04d}'.format(icase) + '.vital'
    df = pd.DataFrame(vf, columns=datatrks) #vf.to_pandas(datatrks, 1 / SRATE) 

    evt1ix, evt2ix = np.nan, np.nan

    try:
        evt1ix = evtdf.loc[evtdf['caseid']==file]['initiation'].values[0]
        evt2ix = evtdf.loc[evtdf['caseid']==file]['completion'].values[0]
    except:
        continue

    if np.isnan(evt1ix)|np.isnan(evt2ix):        
        continue
    
    intudur = (evt2ix-evt1ix)/SRATE    

    # extract heart rate
    pre_hr, pst_hr = np.nan, np.nan
    df[hrcol].loc[(df[hrcol]<30)|(df[hrcol]>180)] = np.nan
    pre_hr = df[hrcol].iloc[evt1ix - SEGLEN:evt1ix].mean(skipna=True)
    pst_hr = df[hrcol].iloc[evt2ix: evt2ix+SEGLEN].mean(skipna=True)
      
      
    # extract noninvasive systolic blood pressure, or continous systolic blood pressure measurement, alternatively.
    pre_bp, pst_bp = np.nan, np.nan
    df['NIBP_SBP'].loc[(df['NIBP_SBP']<40)|(df['NIBP_SBP']>210)] = np.nan
    df['ART_SBP'].loc[(df['ART_SBP']<40)|(df['ART_SBP']>210)] = np.nan
    
    pre_bp = df['NIBP_SBP'].iloc[evt1ix - nibp_window:evt1ix].dropna()
    if len(pre_bp)>0:
        pre_bp = pre_bp.values[-1]
    else:
        pre_bp = df['ART_SBP'].iloc[evt1ix - nibp_window:evt1ix].dropna()
        if len(pre_bp)>0:
            pre_bp = pre_bp.values[-1]
        else:
            pre_bp = np.nan
            
    pst_bp = df['NIBP_SBP'].iloc[evt2ix:evt2ix+nibp_window].dropna()
    if len(pst_bp)>0:
        pst_bp = pst_bp.values[-1]
    else:       
        pst_bp = df['ART_SBP'].iloc[evt2ix:evt2ix+nibp_window].dropna()
        if len(pst_bp)>0:
            pst_bp = pst_bp.values[-1]
        else:
            pst_bp = np.nan

    hr_change = round((pst_hr - pre_hr) / (pre_hr+0.1) * 100, 2)
    bp_change = round((pst_bp - pre_bp) / (pre_bp+0.1) * 100, 2)


    row = {'vitalfile':file, 'evt1': round(evt1ix / SRATE / 60,1), 'evt2': round(evt2ix / SRATE / 60,1), 'intu_dur' : intudur,\
            'hr_change':hr_change, 'bp_change':bp_change}
    prerow = row.copy()
    prerow['hr'] = pre_hr
    prerow['bp'] = pre_bp
    prerow['timing'] = 0
    
    pstrow = row.copy()
    pstrow['hr'] = pst_hr
    pstrow['bp'] = pst_bp    
    pstrow['timing'] = 1

    predf = df.iloc[evt1ix - SEGLEN : evt1ix].mean(skipna=True)
    pstdf = df.iloc[evt2ix : evt2ix + SEGLEN].mean(skipna=True)

    for col in [remicol, ppfcol, 'BIS']:
        try:
            prerow[col]= round(predf[col], 2)
        except: 
            prerow[col]= np.nan

    for col in [remicol, ppfcol, 'BIS']:
        try:
            pstrow[col]= round(pstdf[col],2)
        except: 
            pstrow[col]= np.nan

    df[eegcol].loc[(df[eegcol]>100)|(df[eegcol]<-100)] = np.nan
    
    process_eeg(df[eegcol], evt1ix-prewindow, prewindow, SEGLEN, SLIDE, prerow)
    process_eeg(df[eegcol], evt2ix, pstwindow, SEGLEN, SLIDE, pstrow)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

In [6]:
df

,0,1,2,3,4,5,6,7
0,NaN,NaN,NaN,NaN,NaN,NaN,22.850000,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,22.900000,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,23.049999,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,23.299999,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,23.700001,NaN
...,...,...,...,...,...,...,...,...
1477256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1477257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1477258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1477259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
